# **Image Restoration**

## Setup Environment

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import zipfile

# Environment variables
WIDTH = 128
HEIGHT = 128
QUANTITY = 100
EPOCH_SAVE = 25
BATCH_SIZE = 50

# Test if it's running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:  # If it's running in Google Colab, mount the drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Environment variables
    HOME = "/content/drive/MyDrive/Image Restoration/image-restoration/"
    LOCALHOME = "/content/image-restoration/"
    TRAINING_PATH = LOCALHOME + "Dataset/Training/"
    EVALUATION_PATH = LOCALHOME + "Dataset/Evaluation/"
    BATCH_SIZE = 160
    RESULTS_PATH = HOME + "Results/" # Default HOME. Change this to LOCALHOME to test locally
    MODEL_PATH = HOME + "Model/" # Default HOME. Change this to LOCALHOME to test locally

    os.chdir("/content")
    if not os.path.exists(LOCALHOME): # If the repo isn't cloned yet, clone it
        # Clone the repo
        !git clone https://github.com/davegabe/image-restoration.git
        # Copy the dataset
        with zipfile.ZipFile(HOME + "Dataset/dataset-ready-to-use.zip", "r") as zip_ref:
            zip_ref.extractall(LOCALHOME + "Dataset/")
        os.chdir(LOCALHOME)
    else: # If it's already cloned, update it
        os.chdir(LOCALHOME)
        !git pull
else:  # If it's running in local machine, use the local path
    # Environment variables
    HOME = "./"
    TRAINING_PATH = HOME + "Dataset/Training/"
    EVALUATION_PATH = HOME + "Dataset/Evaluation/"
    RESULTS_PATH = HOME + "Results/"
    MODEL_PATH = HOME + "Model/"
    os.chdir(HOME)

os.makedirs(TRAINING_PATH + "Original/", exist_ok=True)
os.makedirs(TRAINING_PATH + "Corrupted/", exist_ok=True)
os.makedirs(EVALUATION_PATH + "Original/", exist_ok=True)
os.makedirs(EVALUATION_PATH + "Corrupted/", exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

## Data Download

In [ ]:
from data.downloader import adjustAndCrop, splitData
from data.corrupter import corrupt
import shutil

# Test if it's running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

def doYouReallyWantToDownload():
    trainingFiles = len(os.listdir(TRAINING_PATH + "Original/")) + len(os.listdir(TRAINING_PATH + "Original/"))
    evaluationFiles = len(os.listdir(EVALUATION_PATH + "Original/")) + len(os.listdir(EVALUATION_PATH + "Corrupted/"))
    if trainingFiles + evaluationFiles > 0:
        if input("Oops, there seems to be some files already, do you want to overwrite them? (y/n)").capitalize() == "Y":
            # remove all training files
            shutil.rmtree(TRAINING_PATH + "Original/")
            shutil.rmtree(TRAINING_PATH + "Corrupted/")
            shutil.rmtree(EVALUATION_PATH + "Original/")
            shutil.rmtree(EVALUATION_PATH + "Corrupted/")
            os.makedirs(TRAINING_PATH + "Original/", exist_ok=True)
            os.makedirs(TRAINING_PATH + "Corrupted/", exist_ok=True)
            os.makedirs(EVALUATION_PATH + "Original/", exist_ok=True)
            os.makedirs(EVALUATION_PATH + "Corrupted/", exist_ok=True)
            return True
        else:
            return False
    else:
            return True


if IN_COLAB:
    print("Not implemented yet. Upload the \"dataset-ready-to-use.zip\" to the drive manually.")
else:
    print("Generating dataset...")

    if doYouReallyWantToDownload():
        # Unzip the dataset placed inside "./Dataset/landscape-pictures-kaggle.zip"
        with zipfile.ZipFile(HOME + "Dataset/landscape-pictures-kaggle.zip", "r") as zip_ref:
            zip_ref.extractall(TRAINING_PATH + "Original")

        adjustAndCrop(TRAINING_PATH + "Original/", WIDTH, HEIGHT)
        splitData(TRAINING_PATH + "Original/", EVALUATION_PATH + "Original/")
        corrupt(TRAINING_PATH + "Original/", EVALUATION_PATH + "Original/", TRAINING_PATH + "Corrupted/", EVALUATION_PATH + "Corrupted/")
        print("Dataset generated.")
    else:
        print("Dataset not generated (already exists).")


## Training

In [ ]:
from autoencoder.train import train

train(TRAINING_PATH, MODEL_PATH, EPOCH_SAVE, BATCH_SIZE, WIDTH, HEIGHT)

## Testing

In [ ]:
from autoencoder.test import test

TRAINING_PATH = HOME + "Dataset/Training/"
EVALUATION_PATH = HOME + "Dataset/Evaluation/"
test(MODEL_PATH, EVALUATION_PATH, RESULTS_PATH, BATCH_SIZE, WIDTH, HEIGHT)

## Cleanup

In [ ]:
# remove RESULTS_PATH and MODEL_PATH and its contents
import shutil
shutil.rmtree(RESULTS_PATH)
shutil.rmtree(MODEL_PATH)
os.makedirs(RESULTS_PATH, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)